### import or install libraries required

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# !conda install -c conda-forge scikit-learn 
from sklearn.cluster import KMeans


# !pip install folium
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests
import csv
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


### Extract Sydney SubUrbs info

In [2]:
# request
data = requests.get('https://auspost.com.au/postcode/sydney').text

# parsing
soup = BeautifulSoup(data)

postalCodeList = []
subUrb = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    #print(len(cells))
    #print(cells)
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        #print(cells[0].text.rstrip('\n'))
        subUrb.append(cells[1].text.rstrip('\n')) 
        #print(cells[1].text.rstrip('\n'))
    

In [3]:
# create Sydney dataframe
Syd = pd.DataFrame({"Postal Code": postalCodeList,
                           "SubUrb": subUrb})

Syd

,Postal Code,SubUrb
0,2055,"NORTH SYDNEY, NSW"
1,2059,"NORTH SYDNEY, NSW"
2,2060,"NORTH SYDNEY, NSW"
3,2060,"NORTH SYDNEY SHOPPINGWORLD, NSW"
4,2000,"SYDNEY, NSW"
5,2001,"SYDNEY, NSW"
6,2020,"SYDNEY DOMESTIC AIRPORT, NSW"
7,2020,"SYDNEY INTERNATIONAL AIRPORT, NSW"
8,2129,"SYDNEY MARKETS, NSW"
9,2127,"SYDNEY OLYMPIC PARK, NSW"


### Sydney - Remove any Duplicate Postal Codes, given this report is based on postal offices in the area

In [4]:
Syd.sort_values("Postal Code", inplace = True)

Syd.drop_duplicates(subset ="Postal Code", 
                     keep = "first", inplace = True) 

Syd = Syd.reset_index(drop=True)

Syd

,Postal Code,SubUrb
0,1231,"SYDNEY SOUTH, NSW"
1,1235,"SYDNEY SOUTH, NSW"
2,1466,"UNSW SYDNEY, NSW"
3,2000,"SYDNEY, NSW"
4,2001,"SYDNEY, NSW"
5,2006,"THE UNIVERSITY OF SYDNEY, NSW"
6,2020,"SYDNEY DOMESTIC AIRPORT, NSW"
7,2055,"NORTH SYDNEY, NSW"
8,2059,"NORTH SYDNEY, NSW"
9,2060,"NORTH SYDNEY, NSW"


### Extract Melbourne SubUrbs info

In [5]:
# request
data = requests.get('https://auspost.com.au/postcode/melbourne').text

# parsing
soup = BeautifulSoup(data)

postalCodeList = []
subUrb = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    #print(len(cells))
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        subUrb.append(cells[1].text.rstrip('\n'))    


In [6]:
# create Melbourne dataframe
Mel = pd.DataFrame({"Postal Code": postalCodeList,
                           "SubUrb": subUrb})

Mel

,Postal Code,SubUrb
0,3002,"EAST MELBOURNE, VIC"
1,8002,"EAST MELBOURNE, VIC"
2,3000,"MELBOURNE, VIC"
3,3001,"MELBOURNE, VIC"
4,3004,"MELBOURNE, VIC"
5,8001,"MELBOURNE, VIC"
6,3045,"MELBOURNE AIRPORT, VIC"
7,3052,"MELBOURNE UNIVERSITY, VIC"
8,5006,"NORTH ADELAIDE MELBOURNE ST, SA"
9,3051,"NORTH MELBOURNE, VIC"


### Melbourne - Remove any Duplicate Postal Codes, given this report is based on postal offices in the area

In [7]:
Mel.sort_values("Postal Code", inplace = True)

Mel.drop_duplicates(subset ="Postal Code", 
                     keep = "first", inplace = True) 

Mel = Mel.reset_index(drop=True)

Mel

,Postal Code,SubUrb
0,3000,"MELBOURNE, VIC"
1,3001,"MELBOURNE, VIC"
2,3002,"EAST MELBOURNE, VIC"
3,3003,"WEST MELBOURNE, VIC"
4,3004,"MELBOURNE, VIC"
5,3010,"UNIVERSITY OF MELBOURNE, VIC"
6,3045,"MELBOURNE AIRPORT, VIC"
7,3050,"ROYAL MELBOURNE HOSPITAL, VIC"
8,3051,"NORTH MELBOURNE, VIC"
9,3052,"MELBOURNE UNIVERSITY, VIC"


### Create New DataFrames and Populate latitude and longitude for the respective SubUrbs

In [8]:
#test the latitude and longitude details for one address
address = '2000, Sydney, NSW'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-33.869844189970706 151.20720596200005


### New Sydney DataFrame

In [10]:
Syd_new = pd.DataFrame()

for i in range(len(Syd)) : 
    
    try:
        
        #print(Syd.loc[i, "Postal Code"] + ", "+ Syd.loc[i, "SubUrb"])
        address = Syd.loc[i, "Postal Code"] + ", "+ Syd.loc[i, "SubUrb"]

        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        Syd_new.loc[i, "Postal Code"]=Syd.loc[i, "Postal Code"]
        Syd_new.loc[i, "SubUrb"]=Syd.loc[i, "SubUrb"]
        Syd_new.loc[i, "latitude"]=latitude
        Syd_new.loc[i, "longitude"]=longitude

    except:
        
        #print(Syd.loc[i, "Postal Code"] + ", "+ Syd.loc[i, "SubUrb"])
        address = Syd.loc[i, "SubUrb"]

        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        Syd_new.loc[i, "Postal Code"]=Syd.loc[i, "Postal Code"]
        Syd_new.loc[i, "SubUrb"]=Syd.loc[i, "SubUrb"]
        Syd_new.loc[i, "latitude"]=latitude
        Syd_new.loc[i, "longitude"]=longitude        

Syd_new    

,Postal Code,SubUrb,latitude,longitude
0,1231,"SYDNEY SOUTH, NSW",-33.854816,151.216454
1,1235,"SYDNEY SOUTH, NSW",-33.854816,151.216454
2,1466,"UNSW SYDNEY, NSW",-33.917015,151.225182
3,2000,"SYDNEY, NSW",-33.869844,151.207206
4,2001,"SYDNEY, NSW",-36.427889,148.614567
5,2006,"THE UNIVERSITY OF SYDNEY, NSW",-33.888907,151.189434
6,2020,"SYDNEY DOMESTIC AIRPORT, NSW",-33.933728,151.181984
7,2055,"NORTH SYDNEY, NSW",-33.841190,151.207469
8,2059,"NORTH SYDNEY, NSW",-33.841190,151.207469
9,2060,"NORTH SYDNEY, NSW",-33.840468,151.209086


### New Melbourne DataFrame

In [11]:
Mel_new = pd.DataFrame()

for i in range(len(Mel)) : 
    
    try:
        
        #print(Syd.loc[i, "Postal Code"] + ", "+ Syd.loc[i, "SubUrb"])
        address = Mel.loc[i, "Postal Code"] + ", "+ Mel.loc[i, "SubUrb"]

        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        Mel_new.loc[i, "Postal Code"]=Mel.loc[i, "Postal Code"]
        Mel_new.loc[i, "SubUrb"]=Mel.loc[i, "SubUrb"]
        Mel_new.loc[i, "latitude"]=latitude
        Mel_new.loc[i, "longitude"]=longitude

    except:
        
        #print(Mel.loc[i, "Postal Code"] + ", "+ Mel.loc[i, "SubUrb"])
        address = Mel.loc[i, "SubUrb"]

        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        Mel_new.loc[i, "Postal Code"]=Mel.loc[i, "Postal Code"]
        Mel_new.loc[i, "SubUrb"]=Mel.loc[i, "SubUrb"]
        Mel_new.loc[i, "latitude"]=latitude
        Mel_new.loc[i, "longitude"]=longitude        

Mel_new

,Postal Code,SubUrb,latitude,longitude
0,3000,"MELBOURNE, VIC",-37.813562,144.963424
1,3001,"MELBOURNE, VIC",-37.814327,144.962604
2,3002,"EAST MELBOURNE, VIC",-37.810965,144.979667
3,3003,"WEST MELBOURNE, VIC",-37.807744,144.946565
4,3004,"MELBOURNE, VIC",-37.838951,144.975018
5,3010,"UNIVERSITY OF MELBOURNE, VIC",-37.797080,144.961301
6,3045,"MELBOURNE AIRPORT, VIC",-37.675556,144.849071
7,3050,"ROYAL MELBOURNE HOSPITAL, VIC",-37.798669,144.956412
8,3051,"NORTH MELBOURNE, VIC",-37.801557,144.949367
9,3052,"MELBOURNE UNIVERSITY, VIC",-37.797080,144.961301


### Sydney Map

In [13]:
map_sydney = folium.Map(location=[latitude, longitude], zoom_start=9)

# Adding markers to map
for lat, lng, neighborhood in zip(Syd_new['latitude'],  Syd_new['longitude'], Syd_new['SubUrb']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_sydney)

map_sydney

### Melbourne Map

In [14]:
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=9)

# Adding markers to map
for lat, lng, neighborhood in zip(Mel_new['latitude'],  Mel_new['longitude'], Mel_new['SubUrb']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_melbourne)

map_melbourne

### Use the foursquare API to explore the neighbourhoods

In [15]:
CLIENT_ID = 'DBZO4W3ELFXYUARFFR4FI4QIQHECMWYRQUZ0YKHJHM11KETP' # your Foursquare ID
CLIENT_SECRET = 'DBTTURHNNFSMTT3ROBFHM2KV0YFP5AUMDW5Q0MLNFMNIHWHN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBZO4W3ELFXYUARFFR4FI4QIQHECMWYRQUZ0YKHJHM11KETP
CLIENT_SECRET:DBTTURHNNFSMTT3ROBFHM2KV0YFP5AUMDW5Q0MLNFMNIHWHN


### Sydney - extract the venues

In [38]:
radius = 2000
LIMIT = 100
venues = []
for lat, long, neighborhood in zip(Syd_new['latitude'], Syd_new['longitude'], Syd_new['SubUrb']):
# Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
        # print(url) # display URL

        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # Return only relevant information for each nearby venue
        for venue in results:
            venues.append((neighborhood,lat,long,venue['venue']['name'],
            venue['venue']['location']['lat'],venue['venue']['location']    ['lng'],venue['venue']['categories'][0]['name']))

### Melbourne - extract the venues

In [39]:
radius = 2000
LIMIT = 100
venues_m = []
for lat, long, neighborhood in zip(Mel_new['latitude'], Mel_new['longitude'], Mel_new['SubUrb']):
# Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
        # print(url) # display URL

        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # Return only relevant information for each nearby venue
        for venue in results:
            venues_m.append((neighborhood,lat,long,venue['venue']['name'],
            venue['venue']['location']['lat'],venue['venue']['location']    ['lng'],venue['venue']['categories'][0]['name']))

In [43]:
# venues
# venues_m

[('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Sydney Opera House - Concert Hall',
  -33.856594787181294,
  151.21505812823835,
  'Concert Hall'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Sydney Opera House - Studio',
  -33.8569905671763,
  151.21448184016464,
  'Theater'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Drama Theatre',
  -33.8568632128004,
  151.2147593377449,
  'Theater'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Sydney Opera House',
  -33.85726,
  151.21504,
  'Opera House'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Sydney Opera House - Playhouse',
  -33.85751752956304,
  151.21455903252354,
  'Theater'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Opera Kitchen',
  -33.8578529,
  151.2141265,
  'Café'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Opera Bar',
  -33.8584089,
  151.2139761,
  'Cocktail Bar'),
 ('SYDNEY SOUTH, NSW',
  -33.8548157,
  151.2164539,
  'Bennelong Restauran

### Sydney - Venue DataFrame

In [45]:
venues_df = pd.DataFrame(venues)
# Defining the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1119, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"SYDNEY SOUTH, NSW",-33.854816,151.216454,Sydney Opera House - Concert Hall,-33.856595,151.215058,Concert Hall
1,"SYDNEY SOUTH, NSW",-33.854816,151.216454,Sydney Opera House - Studio,-33.856991,151.214482,Theater
2,"SYDNEY SOUTH, NSW",-33.854816,151.216454,Drama Theatre,-33.856863,151.214759,Theater
3,"SYDNEY SOUTH, NSW",-33.854816,151.216454,Sydney Opera House,-33.857260,151.215040,Opera House
4,"SYDNEY SOUTH, NSW",-33.854816,151.216454,Sydney Opera House - Playhouse,-33.857518,151.214559,Theater


### Melbourne - Venue DataFrame

In [46]:
venues_df_m = pd.DataFrame(venues_m)
# Defining the column names
venues_df_m.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df_m.shape)
venues_df_m.head()

(1388, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"MELBOURNE, VIC",-37.813562,144.963424,Tipo 00,-37.813527,144.961978,Italian Restaurant
1,"MELBOURNE, VIC",-37.813562,144.963424,Kirk's Wine Bar,-37.813661,144.961351,Wine Bar
2,"MELBOURNE, VIC",-37.813562,144.963424,Brother Baba Budan,-37.813445,144.962137,Coffee Shop
3,"MELBOURNE, VIC",-37.813562,144.963424,Little Rogue Coffee,-37.810986,144.964059,Coffee Shop
4,"MELBOURNE, VIC",-37.813562,144.963424,B'cos Brazil,-37.815486,144.963085,Brazilian Restaurant


### Sydney Unique Categories

In [47]:
# Lets check how many venues were returned for each neighbourhood
venues_df.groupby(["Neighborhood"]).count()
# Lets check out how many unique categories can be curated from all the returned values
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))
# There are 155 unique categories
# Displaying the first 50 Venue Category names
venues_df['VenueCategory'].unique()[:50]

There are 155 unique categories.


array(['Concert Hall', 'Theater', 'Opera House', 'Café', 'Cocktail Bar',
       'Australian Restaurant', 'Bridge', 'Plaza', 'Scenic Lookout',
       'Trail', 'Monument / Landmark', 'Hotel', 'History Museum',
       'Tour Provider', 'Restaurant', 'Park', 'Burger Joint',
       'Botanical Garden', 'Pub', 'Art Museum', 'Bakery', 'Garden',
       'Hotel Bar', 'Performing Arts Venue', 'Hostel', 'Flea Market',
       'Bar', 'Thai Restaurant', 'Ice Cream Shop', 'Pool', 'Brewery',
       'Lounge', 'Japanese Restaurant', 'Tree', 'Food Court', 'Speakeasy',
       'Italian Restaurant', 'Library', 'Chinese Restaurant',
       'French Restaurant', 'Seafood Restaurant', 'Planetarium',
       'Pizza Place', 'Creperie', 'Sushi Restaurant', 'Steakhouse',
       'Art Gallery', 'Middle Eastern Restaurant',
       'General College & University', 'Indonesian Restaurant'],
      dtype=object)

### Melbourne Unique Categories

In [48]:
# Lets check how many venues were returned for each neighbourhood
venues_df_m.groupby(["Neighborhood"]).count()
# Lets check out how many unique categories can be curated from all the returned values
print('There are {} unique categories.'.format(len(venues_df_m['VenueCategory'].unique())))
# There are 168 unique categories
# Displaying the first 50 Venue Category names
venues_df_m['VenueCategory'].unique()[:50]

There are 168 unique categories.


array(['Italian Restaurant', 'Wine Bar', 'Coffee Shop',
       'Brazilian Restaurant', 'Bar', 'Whisky Bar', 'Cocktail Bar',
       'Japanese Restaurant', 'Shopping Mall', 'Rooftop Bar',
       'Donut Shop', 'Asian Restaurant', 'Indonesian Restaurant',
       'Dessert Shop', 'Theater', 'Hobby Shop', 'Café', 'Library',
       'Thai Restaurant', 'Miscellaneous Shop', 'Pizza Place',
       'Clothing Store', 'Road', 'Movie Theater', 'Bubble Tea Shop',
       'Argentinian Restaurant', 'General Entertainment', 'Creperie',
       'Scandinavian Restaurant', 'Cosmetics Shop', 'Ice Cream Shop',
       'Bagel Shop', 'BBQ Joint', 'Chinese Restaurant', 'Plaza',
       'Comic Shop', 'Boutique', 'Hotel', 'Street Art', 'Bakery',
       'Juice Bar', 'Museum', 'Vietnamese Restaurant',
       'Korean Restaurant', 'Cupcake Shop', 'Peruvian Restaurant',
       'Shanghai Restaurant', 'Speakeasy', 'Pedestrian Plaza',
       'Spanish Restaurant'], dtype=object)

### Sydney Onehot Coding

In [25]:
# One hot encoding
syd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
# Adding neighborhood column back to dataframe
syd_onehot['SubUrb'] = venues_df['Neighborhood']
# Moving neighbourhood column to the first column
fixed_columns = [syd_onehot.columns[-1]] + list(syd_onehot.columns[:-1])
syd_onehot = syd_onehot[fixed_columns]
print(syd_onehot.shape)


(1119, 156)


### Melbourne Onehot Coding

In [50]:
# One hot encoding
mel_onehot = pd.get_dummies(venues_df_m[['VenueCategory']], prefix="", prefix_sep="")
# Adding neighborhood column back to dataframe
mel_onehot['SubUrb'] = venues_df_m['Neighborhood']
# Moving neighbourhood column to the first column
fixed_columns = [mel_onehot.columns[-1]] + list(mel_onehot.columns[:-1])
mel_onehot = mel_onehot[fixed_columns]
print(mel_onehot.shape)


(1388, 169)


### Sydney - Grouped DataFrame

In [28]:
syd_grouped=syd_onehot.groupby(["SubUrb"]).sum().reset_index()
print(syd_grouped.shape)
syd_grouped

(8, 156)


,SubUrb,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,...,Track Stadium,Trail,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Yoga Studio
0,"NORTH SYDNEY, NSW",0,0,0,0,0,0,0,0,3,...,0,6,0,3,0,3,0,0,0,0
1,"SYDNEY DOMESTIC AIRPORT, NSW",1,1,8,2,0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,"SYDNEY MARKETS, NSW",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,3,1,1
3,"SYDNEY OLYMPIC PARK, NSW",0,0,0,0,1,0,0,0,1,...,1,1,0,1,0,3,2,0,0,0
4,"SYDNEY SOUTH, NSW",0,0,0,0,0,0,2,2,0,...,0,2,2,0,0,0,0,0,0,0
5,"SYDNEY, NSW",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,1,0
6,"THE UNIVERSITY OF SYDNEY, NSW",0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,2,1
7,"UNSW SYDNEY, NSW",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### Melbourne Grouped DataFrame

In [51]:
mel_grouped=mel_onehot.groupby(["SubUrb"]).sum().reset_index()
print(mel_grouped.shape)
mel_grouped

(11, 169)


,SubUrb,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,...,Theme Park Ride / Attraction,Track,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yunnan Restaurant,Zoo
0,"EAST MELBOURNE, VIC",0,0,0,0,0,0,0,2,2,...,0,1,0,2,2,6,7,0,0,0
1,"MELBOURNE AIRPORT, VIC",0,2,9,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"MELBOURNE UNIVERSITY, VIC",1,0,0,0,0,0,0,0,1,...,0,1,0,3,1,2,2,1,1,0
3,"MELBOURNE, VIC",0,1,9,2,1,0,0,3,1,...,0,2,0,1,3,6,4,1,0,0
4,"NORTH ADELAIDE MELBOURNE ST, SA",0,0,0,0,0,1,0,0,0,...,0,0,0,2,1,0,2,1,0,1
5,"NORTH MELBOURNE, VIC",1,0,0,0,0,0,1,0,1,...,0,0,0,1,0,2,2,0,1,0
6,"PORT MELBOURNE, VIC",0,0,0,0,0,1,0,0,0,...,0,0,1,1,1,0,0,0,0,0
7,"ROYAL MELBOURNE HOSPITAL, VIC",1,0,0,0,0,0,0,0,1,...,0,1,0,1,0,2,2,0,1,0
8,"SOUTH MELBOURNE, VIC",0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,0
9,"UNIVERSITY OF MELBOURNE, VIC",1,0,0,0,0,0,0,0,1,...,0,1,0,3,1,2,2,1,1,0


### Sydney Top 5 Venues

In [29]:
num_top_venues = 5

for hood in syd_grouped['SubUrb']:
    print("----"+hood+"----")
    temp = syd_grouped[syd_grouped['SubUrb'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----NORTH SYDNEY, NSW----
                 venue  freq
0                 Café  57.0
1                 Park  33.0
2       Scenic Lookout  19.0
3  Japanese Restaurant  18.0
4                  Bar  17.0


----SYDNEY DOMESTIC AIRPORT, NSW----
            venue  freq
0  Airport Lounge   8.0
1            Café   8.0
2           Hotel   8.0
3          Bakery   6.0
4     Coffee Shop   5.0


----SYDNEY MARKETS, NSW----
             venue  freq
0  Thai Restaurant   8.0
1      Coffee Shop   8.0
2             Café   6.0
3              Bar   5.0
4            Hotel   5.0


----SYDNEY OLYMPIC PARK, NSW----
                  venue  freq
0                  Café  20.0
1                  Park   6.0
2                 Hotel   4.0
3   Japanese Restaurant   4.0
4  Fast Food Restaurant   3.0


----SYDNEY SOUTH, NSW----
                   venue  freq
0                   Café  26.0
1         Scenic Lookout  14.0
2  Australian Restaurant  10.0
3                    Pub  10.0
4                   Park  10.0


----SY

### Melbourne Top 5 Venues

In [52]:
num_top_venues = 5

for hood in mel_grouped['SubUrb']:
    print("----"+hood+"----")
    temp = mel_grouped[mel_grouped['SubUrb'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----EAST MELBOURNE, VIC----
                 venue  freq
0                 Café  16.0
1         Cocktail Bar  12.0
2                  Bar   9.0
3  Japanese Restaurant   9.0
4                 Park   9.0


----MELBOURNE AIRPORT, VIC----
                 venue  freq
0       Airport Lounge   9.0
1                 Café   6.0
2          Coffee Shop   3.0
3  Rental Car Location   2.0
4         Noodle House   2.0


----MELBOURNE UNIVERSITY, VIC----
                venue  freq
0                Café  19.0
1         Coffee Shop   8.0
2                 Bar   6.0
3   Korean Restaurant   3.0
4  Italian Restaurant   3.0


----MELBOURNE, VIC----
            venue  freq
0            Café  39.0
1             Bar  22.0
2     Coffee Shop  20.0
3    Cocktail Bar  13.0
4  Airport Lounge   9.0


----NORTH ADELAIDE MELBOURNE ST, SA----
             venue  freq
0             Café   6.0
1              Pub   5.0
2      Coffee Shop   4.0
3            Hotel   4.0
4  Thai Restaurant   4.0


----NORTH MELBOURNE, VIC

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Sydney Top 10 Venues Names List

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['SubUrb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
SubUrbs_venues_sorted = pd.DataFrame(columns=columns)
SubUrbs_venues_sorted['SubUrb'] = syd_grouped['SubUrb']

for ind in np.arange(syd_grouped.shape[0]):
    SubUrbs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(syd_grouped.iloc[ind, :], num_top_venues)

SubUrbs_venues_sorted.head()

,SubUrb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"NORTH SYDNEY, NSW",Café,Park,Scenic Lookout,Japanese Restaurant,Bar,Theater,Australian Restaurant,Italian Restaurant,Coffee Shop,Trail
1,"SYDNEY DOMESTIC AIRPORT, NSW",Hotel,Airport Lounge,Café,Bakery,Coffee Shop,Fast Food Restaurant,Juice Bar,Thai Restaurant,Donut Shop,Sports Bar
2,"SYDNEY MARKETS, NSW",Coffee Shop,Thai Restaurant,Café,Bar,Hotel,Theater,Cocktail Bar,Pub,Japanese Restaurant,Park
3,"SYDNEY OLYMPIC PARK, NSW",Café,Park,Japanese Restaurant,Hotel,Fast Food Restaurant,Athletics & Sports,Supermarket,Vietnamese Restaurant,Australian Restaurant,Seafood Restaurant
4,"SYDNEY SOUTH, NSW",Café,Scenic Lookout,Australian Restaurant,Hotel,Park,Pub,Theater,Hotel Bar,Speakeasy,Pool


### Melbourne Top 10 Venues Names List

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['SubUrb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
SubUrbs_venues_sorted_m = pd.DataFrame(columns=columns)
SubUrbs_venues_sorted_m['SubUrb'] = mel_grouped['SubUrb']

for ind in np.arange(mel_grouped.shape[0]):
    SubUrbs_venues_sorted_m.iloc[ind, 1:] = return_most_common_venues(mel_grouped.iloc[ind, :], num_top_venues)

SubUrbs_venues_sorted_m.head()

,SubUrb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"EAST MELBOURNE, VIC",Café,Cocktail Bar,Japanese Restaurant,Park,Bar,Asian Restaurant,Coffee Shop,Australian Restaurant,Wine Bar,Italian Restaurant
1,"MELBOURNE AIRPORT, VIC",Airport Lounge,Café,Coffee Shop,Hotel,Noodle House,Rental Car Location,Fast Food Restaurant,Juice Bar,Donut Shop,Scenic Lookout
2,"MELBOURNE UNIVERSITY, VIC",Café,Coffee Shop,Bar,Bakery,Korean Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Bookstore,Japanese Restaurant
3,"MELBOURNE, VIC",Café,Bar,Coffee Shop,Cocktail Bar,Airport Lounge,Italian Restaurant,Asian Restaurant,Park,Japanese Restaurant,Pizza Place
4,"NORTH ADELAIDE MELBOURNE ST, SA",Café,Pub,Coffee Shop,Thai Restaurant,Hotel,Pizza Place,Burger Joint,French Restaurant,Bar,Italian Restaurant


### Sydney Clustering

In [35]:
# set number of clusters
kclusters = 4

syd_grouped_clustering = syd_grouped.drop('SubUrb', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(syd_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 0, 3, 2, 0, 0, 3])

In [36]:
# add clustering labels
SubUrbs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

syd_merged = Syd_new

# merge syd_grouped with syd_data to add latitude/longitude for each neighborhood
syd_merged = syd_merged.join(SubUrbs_venues_sorted.set_index('SubUrb'), on='SubUrb')

syd_merged.head() # check the last columns!

,Postal Code,SubUrb,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1231,"SYDNEY SOUTH, NSW",-33.854816,151.216454,2,Café,Scenic Lookout,Australian Restaurant,Hotel,Park,Pub,Theater,Hotel Bar,Speakeasy,Pool
1,1235,"SYDNEY SOUTH, NSW",-33.854816,151.216454,2,Café,Scenic Lookout,Australian Restaurant,Hotel,Park,Pub,Theater,Hotel Bar,Speakeasy,Pool
2,1466,"UNSW SYDNEY, NSW",-33.917015,151.225182,3,Café,Thai Restaurant,Indonesian Restaurant,Chinese Restaurant,Sandwich Place,Convenience Store,Bar,Coffee Shop,Malay Restaurant,Dessert Shop
3,2000,"SYDNEY, NSW",-33.869844,151.207206,0,Café,Coffee Shop,Hotel,Speakeasy,Cocktail Bar,Shopping Mall,Bar,Bakery,Hotel Bar,Thai Restaurant
4,2001,"SYDNEY, NSW",-36.427889,148.614567,0,Café,Coffee Shop,Hotel,Speakeasy,Cocktail Bar,Shopping Mall,Bar,Bakery,Hotel Bar,Thai Restaurant


### Melbourne Clustering

In [55]:
# set number of clusters
kclusters = 4

mel_grouped_clustering = mel_grouped.drop('SubUrb', 1)

# run k-means clustering
kmeans_m = KMeans(n_clusters=kclusters, random_state=0).fit(mel_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_m.labels_[0:10] 

array([2, 3, 1, 0, 3, 1, 3, 1, 3, 1])

In [56]:
# add clustering labels
SubUrbs_venues_sorted_m.insert(0, 'Cluster Labels', kmeans_m.labels_)

mel_merged = Mel_new

# merge syd_grouped with syd_data to add latitude/longitude for each neighborhood
mel_merged = mel_merged.join(SubUrbs_venues_sorted_m.set_index('SubUrb'), on='SubUrb')

mel_merged.head() # check the last columns!

,Postal Code,SubUrb,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3000,"MELBOURNE, VIC",-37.813562,144.963424,0,Café,Bar,Coffee Shop,Cocktail Bar,Airport Lounge,Italian Restaurant,Asian Restaurant,Park,Japanese Restaurant,Pizza Place
1,3001,"MELBOURNE, VIC",-37.814327,144.962604,0,Café,Bar,Coffee Shop,Cocktail Bar,Airport Lounge,Italian Restaurant,Asian Restaurant,Park,Japanese Restaurant,Pizza Place
2,3002,"EAST MELBOURNE, VIC",-37.810965,144.979667,2,Café,Cocktail Bar,Japanese Restaurant,Park,Bar,Asian Restaurant,Coffee Shop,Australian Restaurant,Wine Bar,Italian Restaurant
3,3003,"WEST MELBOURNE, VIC",-37.807744,144.946565,1,Café,Coffee Shop,Korean Restaurant,Japanese Restaurant,Bar,Cocktail Bar,Chinese Restaurant,Italian Restaurant,Korean BBQ Restaurant,Donut Shop
4,3004,"MELBOURNE, VIC",-37.838951,144.975018,0,Café,Bar,Coffee Shop,Cocktail Bar,Airport Lounge,Italian Restaurant,Asian Restaurant,Park,Japanese Restaurant,Pizza Place


### Sydney Clusters on Map

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(syd_merged['latitude'], syd_merged['longitude'], syd_merged['SubUrb'], syd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Melbourne Clusters on Map

In [58]:
# create map
map_clusters_m = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mel_merged['latitude'], mel_merged['longitude'], mel_merged['SubUrb'], mel_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_m)
       
map_clusters_m

In [62]:
print(len(syd_merged.loc[syd_merged['Cluster Labels'] == 0]))
print(len(syd_merged.loc[syd_merged['Cluster Labels'] == 1]))
print(len(syd_merged.loc[syd_merged['Cluster Labels'] == 2]))
print(len(syd_merged.loc[syd_merged['Cluster Labels'] == 3]))

4
3
2
3


In [63]:
print(len(mel_merged.loc[mel_merged['Cluster Labels'] == 0]))
print(len(mel_merged.loc[mel_merged['Cluster Labels'] == 1]))
print(len(mel_merged.loc[mel_merged['Cluster Labels'] == 2]))
print(len(mel_merged.loc[mel_merged['Cluster Labels'] == 3]))

4
5
2
4
